In [1]:
# example

states = ('rainy', 'sunny')
symbols = ('walk', 'shop', 'clean')

start_prob = {
    'rainy': 0.5,
    'sunny': 0.5
}

trans_prob = {
    'rainy': { 'rainy': 0.7, 'sunny': 0.3 },
    'sunny': { 'rainy': 0.4, 'sunny': 0.6 },
}

emit_prob = {
    'rainy': { 'walk': 0.1, 'shop': 0.4, 'clean': 0.5 },
    'sunny': { 'walk': 0.6, 'shop': 0.3, 'clean': 0.1 },
}

sequence = ['walk', 'shop', 'clean', 'clean', 'walk', 'walk', 'walk', 'clean']
# model = Model(states, symbols, start_prob, trans_prob, emit_prob)

# print(model.evaluate(sequence))
# print(model.decod(sequence))

In [2]:
def _normalize_prob(prob, item_set):
    result = {}
    if prob is None:
        number = len(item_set)
        for item in item_set:
            result[item] = 1.0 / number
    else:
        prob_sum = 0.0
        for item in item_set:
            prob_sum += prob.get(item, 0)
            
        if prob_sum > 0:
            for item in item_set:
                result[item] = prob.get(item, 0) / prob_sum
        else:
            for item in item_set:
                result[item] = 0
    
    return result
                
def _normalize_prob_two_dim(prob, item_set1, item_set2):
    result = {}
    if prob is None:
        for item in item_set1:
            result[item] = _normalize_prob(None, item_set2)
    else:
        for item in item_set1:
            result[item] = _normalize_prob(prob.get(item), item_set2)
        
    return result

In [25]:
class Model(object):
    
    def __init__(self, states, symbols, start_prob=None, trans_prob=None, emit_prob=None):
        self._states = set(states)
        self._symbols = set(symbols)
        self._start_prob = _normalize_prob(start_prob, self._states)
        self._trans_prob = _normalize_prob_two_dim(trans_prob, self._states, self._states)
        self._emit_prob = _normalize_prob_two_dim(emit_prob, self._states, self._symbols)
        
    def __repr__(self):
        return "{name}({_states}, {_symbols}, {_start_prob}, {_trans_prob}, {_emit_prob})".format(name=self.__class__.__name__, **self.__dict__)
    
    def states(self):
        """ Return the state set of this model. """
        return set(self._states)
    
    def states_number(self):
        """ Return the number of staes. """
        return len(self._states)
    
    def symbols(self):
        """ Return the symbol set of this model. """
        return set(self._symbols)
    
    def symbols_nubmer(self):
        """ Return the number of symbols. """
        return len(self._symbols)
    
    def start_prob(self, state):
        """ 
        Return the start probability of the given state.
        
        If 'state' is not contained in the state set of this model, 0 is returned.
        """
        if state not in self._states:
            return 0
        return self._start_prob[state]
    
    def trans_prob(self, state_from, state_to):
        """
        Return the probability that transition from state 'state_from' to
        state 'state_to'.
        
        If either the 'state_from' or the 'state_to' are not contained in the
        state set of this model, 0 is returned.
        """
        if state_from not in self._states or state_to not in self._states:
            return 0
        return self._trans_prob[state_from][state_to]
    
    def emit_prob(self, state, symbol):
        """
        Return the emission probability for 'symbol' associated with the 'state'
        
        If either the 'state' or the 'symbol' are not contained in this model,
        0 is returned.
        """
        if state not in self._states or symbol not in self._symbols:
            return 0
        return self._emit_prob[state][symbol]
    
    def _forward(self, sequence):
        sequence_length = len(sequence)
        if sequence_length == 0:
            return []
        
        # initial step 1
        alpha = [{}]
        for state in self._states:
            alpha[0][state] = self.start_prob(state) * self.emit_prob(state, sequence[0])
            
        # recursion from 2 to T
        for index in range(1, sequence_length):
            alpha.append({})
            for state_to in self._states:
                prob = 0
                for state_from in self._states:
                    prob += alpha[index - 1][state_from] * \
                        self.trans_prob(state_from, state_to)
                alpha[index][state_to] = prob * self.emit_prob(state_to, sequence[index])
                
        return alpha
    
    def _backward(self, sequence):
        sequence_length = len(sequence)
        if sequence_length == 0:
            return []
        
        # initial step T
        beta = [{}]
        for state in self._states:
            beta[0][state] = 1
        
        # recursion from T-1 to 1
        for index in range(sequence_length - 1, 0, -1):
            beta.insert(0, {})
            for state_from in self._states:
                prob = 0
                for state_to in self._states:
                    prob += beta[1][state_to] * \
                        self.trans_prob(state_from, state_to) * \
                        self.emit_prob(state_to, sequence[index])
                beta[0][state_from] = prob
                
        return beta
    
    def evaluate(self, sequence):
        length = len(sequence)
        if length == 0:
            return 0
        
        prob = 0
        alpha = self._forward(sequence)
        for state in alpha[length -1]:
            prob += alpha[length -1][state]

        return prob
    
    def decode(self, sequence):
        sequence_length = len(sequence)
        if sequence_length == 0:
            return []
        
        # initial prob for Viterbi time 1
        delta = {}
        for state in self._states:
            delta[state] = self.start_prob(state) * self.emit_prob(state, sequence[0])
        
        # recursion Viterbi from 2 to T
        pre = []
        for index in range(1, sequence_length):
            delta_bar = {}
            pre_state = {}
            for state_to in self._states:
                max_prob = 0
                max_state = None
                for state_from in self._states:
                    prob = delta[state_from] * self.trans_prob(state_from, state_to)
                    if prob > max_prob:
                        max_prob = prob
                        max_state = state_from
                delta_bar[state_to] = max_prob * self.emit_prob(state_to, sequence[index])
                pre_state[state_to] = max_state
            delta = delta_bar
            pre.append(pre_state)

        # the highest probability
        max_state = None
        max_prob = 0
        # get most probable state and its backtrack
        for state in self._states:
            if delta[state] > max_prob:
                max_prob = delta[state]
                max_state = state
        if max_state is None:
            return []
        
        result = [max_state]
        # follow the backtrack till the first observation
        for index in range(sequence_length - 1, 0, -1):
            max_state = pre[index -1][max_state]
            result.insert(0, max_state)

        return result, delta, pre
    
    def learn(self, sequence, smoothing=0):
        """
        Use the given 'sequence' to find the best state transition and
        emission probabilities.
        """
        length = len(sequence)
        alpha = self._forward(sequence)
        beta = self._backward(sequence)
        
        gamma = []
        for index in range(length):
            prob_sum = 0
            gamma.append({})
            for state in self._states:
                prob = alpha[index][state] * beta[index][state]
                gamma[index][state] = prob
                prob_sum += prob
                
            if prob_sum == 0:
                continue
                
            for state in self._states:
                gamma[index][state] /= prob_sum
        
        xi = []
        for index in range(length -1):
            prob_sum = 0
            xi.append({})
            for state_from in self._states:
                xi[index][state_from] = {}
                for state_to in self._states:
                    prob = alpha[index][state_from] * beta[index + 1][state_to] * \
                        self.trans_prob(state_from, state_to) * \
                        self.emit_prob(state_to, sequence[index + 1])
                    xi[index][state_from][state_to] = prob
                    prob_sum += prob
                    
            if prob_sum == 0:
                continue
                
            for state_from in self._states:
                for state_to in self._states:
                    xi[index][state_from][state_to] /= prob_sum
                                        
        states_number = len(self._states)
        symbols_number = len(self._symbols)
        for state in self._states:
            # update start probability
            self._start_prob[state] = (smoothing + gamma[0][state]) / (1 + states_number * smoothing)
            
            # update transition probability
            gamma_sum = 0
            for index in range(length -1):
                gamma_sum += gamma[index][state]
                
            if gamma_sum > 0:
                denominator = gamma_sum + states_number * smoothing
                for state_to in self._states:
                    xi_sum = 0
                    for index in range(length -1):
                        xi_sum += xi[index][state][state_to]
                    self._trans_prob[state][state_to] = (smoothing + xi_sum) / denominator
            else:
                for state_to in self._states:
                    self._trans_prob[state][state_to] = 0
                    
            # update emission probability
            gamma_sum += gamma[length -1][state]
            emit_gamma_sum = {}
            for symbol in self._symbols:
                emit_gamma_sum[symbol] = 0
                
            for index in range(length):
                emit_gamma_sum[sequence[index]] += gamma[index][state]
                
            if gamma_sum > 0:
                denominator = gamma_sum + symbols_number * smoothing
                for symbol in self._symbols:
                    self._emit_prob[state][symbol] = (smoothing + emit_gamma_sum[symbol]) / denominator
            else:
                for symbol in self._symbols:
                    self._emit_prob[state][symbol] = 0

In [27]:
model = Model(states, symbols, start_prob, trans_prob, emit_prob)

In [28]:
# forward result
model._forward(sequence)

[{'sunny': 0.3, 'rainy': 0.05},
 {'sunny': 0.058499999999999996, 'rainy': 0.062},
 {'sunny': 0.00537, 'rainy': 0.0334},
 {'sunny': 0.0013242, 'rainy': 0.012764},
 {'sunny': 0.0027742319999999997, 'rainy': 0.0009464479999999999},
 {'sunny': 0.00116908416, 'rainy': 0.00017722064},
 {'sunny': 0.0004527700127999999, 'rainy': 5.916881120000001e-05},
 {'sunny': 2.8941265103999994e-05, 'rainy': 0.00011126308647999998}]

In [29]:
# backward result
model._backward(sequence)

[{'sunny': 0.00038847986191999995, 'rainy': 0.00047320786015999993},
 {'sunny': 0.000918363992, 'rainy': 0.0013948396459999997},
 {'sunny': 0.0028306312, 'rainy': 0.0037426305999999996},
 {'sunny': 0.01614632, 'rainy': 0.00930926},
 {'sunny': 0.042104, 'rainy': 0.024722},
 {'sunny': 0.10880000000000001, 'rainy': 0.07339999999999999},
 {'sunny': 0.26, 'rainy': 0.38},
 {'sunny': 1, 'rainy': 1}]

In [30]:
# decoding result
model.decode(sequence)

(['sunny', 'rainy', 'rainy', 'rainy', 'sunny', 'sunny', 'sunny', 'rainy'],
 {'sunny': 8.230118399999997e-06, 'rainy': 2.743372799999999e-05},
 [{'sunny': 'sunny', 'rainy': 'sunny'},
  {'sunny': 'sunny', 'rainy': 'rainy'},
  {'sunny': 'rainy', 'rainy': 'rainy'},
  {'sunny': 'rainy', 'rainy': 'rainy'},
  {'sunny': 'sunny', 'rainy': 'sunny'},
  {'sunny': 'sunny', 'rainy': 'sunny'},
  {'sunny': 'sunny', 'rainy': 'sunny'}])

In [39]:
# learn trans_prob result
model.learn(sequence)

In [40]:
model._trans_prob

{'sunny': {'sunny': 0.4909134950741977, 'rainy': 0.5090865049258023},
 'rainy': {'sunny': 0.36344939035771123, 'rainy': 0.6365506096422888}}

In [ ]:
model = Model(states, symbols, start_prob, trans_prob, emit_prob)

In [ ]:
model.decode(sequence)

In [ ]:
model.learn(sequence)

In [ ]:
model.decode(sequence)

In [ ]:
model.evaluate(sequence)

In [ ]:
states, symbols

In [ ]:
emit_prob

In [ ]:
trans_prob

In [ ]:
for state in states:
    print(state)
    for symbol in symbols:
        print(symbol, model.emit_prob(state, symbol))

In [ ]:
for state_from in states:
    print("from ", state_from)
    for state_to in states:
        print("to ", state_to, model.trans_prob(state_from, state_to))